In [68]:
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Load and clean data
df = pd.read_csv(Path("../Data/output.csv"))
df = df.rename(columns={"closing_price$": "closing_price", "date": "date"})
df["date"] = pd.to_datetime(df["date"])
df["closing_price"] = df["closing_price"].astype(float)


In [76]:
# ---- Create features (lagged prices) ----
def create_lags(data, lags=5, feature = ''):
    df_feat = data.copy()
    for lag in range(1, lags + 1):
        df_feat[f"{feature[0:3]}lag_{lag}"] = df_feat[feature].shift(lag)
    return df_feat

In [ ]:
# Remove rows with null values due to value shifts
df_clolag = create_lags(df, lags=5, feature = 'closing_price').dropna().reset_index(drop=True)

In [78]:
df_clolag.head(1)

,date,closing_price,clolag_1,clolag_2,clolag_3,clolag_4,clolag_5
0,2023-01-30,109.76,109.85,108.83,108.05,108.61,108.66


## Creating  lags with prior difference.

Linear regression often works better with stationary time-series. To make the time series staionary the difference between closing stock price with the closing stock price of previous day can be used. By taking this difference trend component of the time series can be removed. Out of the the three baseline models namely 1. Average of five days ( 5 day moving average) 2. The previous day value and 3.Extrapolation method the second method was performed best as it gave minimum error. This is another reason that taking the difference between two days and use that as the target variable in linerar regression model.   

In [ ]:
# Create price difference column in a new dataframe
df_prelag = df_clolag[['date', 'closing_price', 'clolag_1']].copy()
df_prelag["price_diff"] = df_prelag["closing_price"] - df_prelag["clolag_1"]
df_prelag.head(1)

,date,closing_price,clolag_1,price_diff
0,2023-01-30,109.76,109.85,-0.09


In [74]:
df_prelag = create_lags(df_prelag, lags=5, feature = 'price_diff').dropna().reset_index(drop=True)
df_prelag.head(6)

,date,closing_price,clolag_1,price_diff,prilag_1,prilag_2,prilag_3,prilag_4,prilag_5
0,2023-02-06,110.75,111.15,-0.40,1.42,-0.42,0.09,0.30,-0.09
1,2023-02-07,110.30,110.75,-0.45,-0.40,1.42,-0.42,0.09,0.30
2,2023-02-08,110.32,110.30,0.02,-0.45,-0.40,1.42,-0.42,0.09
3,2023-02-09,110.21,110.32,-0.11,0.02,-0.45,-0.40,1.42,-0.42
4,2023-02-10,109.95,110.21,-0.26,-0.11,0.02,-0.45,-0.40,1.42
5,2023-02-13,109.30,109.95,-0.65,-0.26,-0.11,0.02,-0.45,-0.40
